<a href="https://colab.research.google.com/github/celinagacias/dc-symmetron/blob/main/Data%20Cleaning%20and%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, drive
from oauth2client.client import GoogleCredentials

# Data Read-In

In [ ]:
#Authenticate user
auth.authenticate_user()
gauth = GoogleAuth()

#Mount GDrive
gdrive = GoogleDrive(gauth)
gauth.credentials = GoogleCredentials.get_application_default()
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read data from csv
folder = '1iLA5sSyu7ere1AN40TWRf6O6ma5LoIDl'
filename = 'raw_AACT_data.csv'

gfile_list = gdrive.ListFile({'q': "'" +  folder + "'" + " in parents and trashed=false"}).GetList()
gfile = [f for f in gfile_list if f['title'] == filename][0]
df_file = gdrive.CreateFile({'id': gfile['id']})
df_file.GetContentFile(gfile['title'])
df = pd.read_csv(filename)
display(df.head(2))

,nct_id,start_date,start_date_type,study_type,phase,overall_status,last_known_status,population,gender_description,minimum_age,maximum_age,gender,healthy_volunteers,countries_removed,countries_retained,conditions_table,mesh_term,criteria
0,NCT00000113,1997-09-30,NaN,Interventional,Phase 3,Completed,NaN,NaN,NaN,6 Years,12 Years,All,No,{},"{""United States""}",myopia,"eye diseases, refractive errors, myopia",Children between the ages of 6 and 12 years wi...
1,NCT00000114,1984-05-31,NaN,Interventional,Phase 3,Completed,NaN,NaN,NaN,18 Years,49 Years,All,NaN,NaN,NaN,retinitis pigmentosa,"retinitis, eye diseases, hereditary, retinal d...",Men and nonpregnant women between ages 18 and ...


In [ ]:
n = len(df)
print(f'There are {n} studies in our sample.')

There are 42275 studies in our sample.


# Data Cleaning

## Dates

In [ ]:
# convert start date to date
df['start_date'] = df['start_date'].apply(
    lambda x: datetime.strptime(x, '%Y-%m-%d').date() if str(x) != 'nan' else None)

In [ ]:
# extract year into separate column
df['start_year'] = df['start_date'].apply(lambda x: int(x.year) if x is not None else None)

## Countries

In [ ]:
def process_countries(l):
  # converts string of list of countries into lists
  l = l.replace('{','').replace('}','').split(',')
  l = set([c.replace('"','') for c in l])
  return l

In [ ]:
# apply the function to each list of countries
df['countries_removed'] = df['countries_removed'].apply(
    lambda x: process_countries(x) if str(x) != 'nan' else None)

df['countries_retained'] = df['countries_retained'].apply(
    lambda x: process_countries(x) if str(x) != 'nan' else None)

## Ages

In [ ]:
def parse_age(a):
  # Convert text of age into years
  if str(a) == 'nan':
    return np.nan

  num = int(str(a).split(' ')[0])
  if 'Year' in a:
    age = num
  elif 'Month' in a:
    age = num / 12
  elif 'Week' in a:
    age = num / 52
  elif 'Day':
    age = num / 365
  elif 'Hour' in a:
    age = num / (24 * 365)
  elif 'Minute' in a:
    age = num / (24 * 365 * 60)
  return age

df['minimum_age'] = df['minimum_age'].apply(parse_age)
df['maximum_age'] = df['maximum_age'].apply(parse_age)

## Inclusion/Exclusion Criteria

In [ ]:
# identify which studies have clear inclusion vs. exclusion
df['criteria_norm'] = df.criteria.fillna('').str.upper().str.strip().str.replace(' CRITERIA','')
inc_cond = df.criteria_norm.str.contains('INCLUSION:')
exc_cond = df.criteria_norm.str.contains('EXCLUSION:')
parse_mask = (inc_cond) & (exc_cond)
len_parse = df[parse_mask].shape[0]
print(f'There are {len_parse} ({np.round(len_parse*100/n, 2)}%) studies with clearly delineated inclusion/exclusion criteria')

There are 38208 (90.38%) studies with clearly delineated inclusion/exclusion criteria


In [ ]:
def parse_criteria(c):
  '''Splits criteria into inclusion and exclusion'''

  def clean_criteria(s):
    # sub-function to clean up criteria text
    s = s.replace('INCLUSION:','').strip() # remove inclusion title
    s = re.sub(r'(\n\s*)+\n+', '\n', s) # reduce newlines
    return s

  # create list with [inclusion, exclusion]
  c_list = c.split('EXCLUSION:')
  c_list = [clean_criteria(s) for s in c_list]
  return c_list

# apply the function to derive inclusion and exclusion
df['inclusion'] = None
df['exclusion'] = None
df.loc[parse_mask, 'criteria_split'] = df.loc[parse_mask, 'criteria_norm'].apply(parse_criteria)
df.loc[parse_mask, 'inclusion'] = df.loc[parse_mask, 'criteria_split'].apply(lambda x: x[0])
df.loc[parse_mask, 'exclusion'] = df.loc[parse_mask, 'criteria_split'].apply(lambda x: x[1])

## Subsetting

In [ ]:
# remove studies that were paused or suspended
suspend_mask = (df.overall_status.isin(['Terminated', 'Withdrawn', 'Suspended']))
n_suspend = df[suspend_mask].shape[0]
print(f'There are {n_suspend} studies that need to be removed because they are either terminated, withdrawn, or suspended.')
df = df[~suspend_mask].copy(deep=True)
print(f'There are {len(df)} studies remaining.')

There are 4597 studies that need to be removed because they are either terminated, withdrawn, or suspended.
There are 37678 studies remaining.


# Descriptives

In [ ]:
raise Exception("scratch code below")

Exception: ignored

In [ ]:
# obtain subset of cancer studies
cancer = df[(df.mesh_term.str.contains('neoplasm')) | (df.mesh_term.str.contains('cancer'))]

# show most frequent mesh term phrases
cancer['mesh_list'] = cancer.mesh_term.apply(lambda x: list(set(x.split(', '))))

mesh_frequency = pd.Series([i for s in cancer['mesh_list'] for i in s]).value_counts()
display(mesh_frequency.head(40))

<ipython-input-10-df69a8d55b2c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cancer['mesh_list'] = cancer.mesh_term.apply(lambda x: list(set(x.split(', '))))


neoplasms                        8298
neoplasms by site                5611
neoplasms by histologic type     3676
carcinoma                        2177
glandular and epithelial         1750
digestive system diseases        1710
digestive system neoplasms       1655
skin diseases                    1247
urogenital neoplasms             1232
immune system diseases           1163
gastrointestinal diseases        1163
gastrointestinal neoplasms       1118
breast diseases                  1101
breast neoplasms                 1100
lymphoproliferative disorders    1057
immunoproliferative disorders    1056
respiratory tract diseases        977
genital neoplasms                 970
respiratory tract neoplasms       959
thoracic neoplasms                957
lung diseases                     942
lung neoplasms                    941
pathologic processes              929
bronchogenic                      784
bronchial neoplasms               784
lymphatic diseases                779
adenocarcino